# Description
 This solution is based on efficientnet-b3 (net was taken from https://github.com/lukemelas/EfficientNet-PyTorch).
 
 The model was trained in the following order:
 
 Criterion -MSELoss, optimizer - Adam

2 epoch - Learning rate - 0.001

4 epoch - Learning rate - 0.0005

4 epoch - Learning rate - 0.0001

4 epoch - Learning rate - 0.00005

4 epoch - Learning rate - 0.00001

2 epoch - Learning rate - 0.000005

2 epoch - Learning rate - 0.000001

Train took a long time on Colab. Due to limit restriction, model weights were saved on some checkpoint and after that training continued on another account.
Final model weights were saved in file.



In [ ]:
import torch
workDir='/home/data/'
#imSize=224
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


%cd '/home'
!mkdir 'data'
%cd '/home/data'


Mounted at /content/drive
/home
/home/data


# Download data

In [ ]:
!pip install --upgrade fastai
!pip install -U aicrowd-cli

     |████████████████████████████████| 204kB 6.8MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 204kB 52.0MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
     |████████████████████████████████| 174kB 57.8MB/s 
     |████████████████████████████████| 81kB 13.0MB/s 
     |████████████████████████████████| 92kB 16.4MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
   

In [ ]:
API_KEY = '52ab6eb031245b7028158e2f3e993174' #Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [ ]:
!aicrowd dataset download --challenge f1-speed-recognition

sample_submission.csv: 100% 97.8k/97.8k [00:00<00:00, 1.59MB/s]
train.csv: 100% 407k/407k [00:00<00:00, 3.67MB/s]
train.zip:   0% 0.00/385M [00:00<?, ?B/s]
train.zip:  78% 302M/385M [00:03<00:00, 85.5MB/s]
train.zip: 100% 385M/385M [00:04<00:00, 83.2MB/s]
val.csv: 100% 36.7k/36.7k [00:00<00:00, 1.46MB/s]

val.zip:   0% 0.00/37.8M [00:00<?, ?B/s]
test.zip: 100% 96.9M/96.9M [00:07<00:00, 13.5MB/s]
val.zip: 100% 37.8M/37.8M [00:03<00:00, 11.8MB/s]


In [ ]:
!rm -rf data
!mkdir data

!unzip -q train.zip  -d data/train
!unzip -q val.zip -d data/val
!unzip -q test.zip  -d data/test

!mv train.csv data/train.csv
!mv val.csv data/val.csv
!mv sample_submission.csv data/sample_submission.csv

# Prepare data

In [ ]:
# Custom dataset class
import torch
from torch.utils.data import Dataset,DataLoader,RandomSampler
from torchvision import transforms as T
import pandas as pd
from PIL import Image

class ImageDataset(Dataset):
  def __init__(self,ImageFold,df,transforms):
    self.ImageFold=ImageFold
    self.df=df
    self.trans=transforms

  def __len__(self):
    return len(self.df)

  def __getitem__(self,ind):
    im=self.load_image(self.df.iloc[ind][0])
    speed=self.df.iloc[ind][1]
    im=self.trans(im)
    return im, speed


  def load_image(self,ind):
    return Image.open(self.ImageFold+str(self.df.iloc[ind][0])+'.jpg')

In [ ]:
trainTrans=T.Compose([
#        T.Resize(imSize),
#        transforms.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])

])

df_train=pd.read_csv('data/train.csv')
ds_train=ImageDataset(workDir+'data/train/',df_train,trainTrans)
dl_train=DataLoader(ds_train,batch_size=32,shuffle=True,num_workers=4)

df_val=pd.read_csv('data/val.csv')
ds_val=ImageDataset(workDir+'data/val/',df_val,trainTrans)
dl_val=DataLoader(ds_val,batch_size=32,shuffle=True,num_workers=4)

dataloaders_dict={'train':dl_train,'val':dl_val}

# Training loop

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_loss = 10e30

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            i=0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    i+=128

                    outputs = model(inputs)
                    loss = criterion(outputs.float(), torch.reshape(labels,(len(labels),1)).float())

                    if(i % 8192 ==0):
                      print(loss)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.detach().item()*len(labels)
            epoch_loss = running_loss / (len(dataloaders[phase].dataset))

            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_loss)
                # statistics

            print('{} Loss: {:.4f} '.format(phase, epoch_loss))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

# EfficientNet training

In [ ]:
import random

import torchvision.models as models
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=b39755b3dc041d8a60548dd1479d328053a7914bca3a8eed7d08b1e9ae9dc035
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b5',num_classes = 1)
model.load_state_dict(torch.load('/content/drive/MyDrive/weights_ef5_net_adam4.txt'))
model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth



Loaded pretrained weights for efficientnet-b5


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [ ]:
criterion = nn.MSELoss(reduction='mean')

In [ ]:
num_epochs=3
optimizer =torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.MSELoss(reduction='mean')
# Train and evaluate
model, hist = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef_net_adam1.txt')

Epoch 0/2
----------
tensor(1126039.3750, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(928345., device='cuda:0', grad_fn=<MseLossBackward>)
tensor(936889.3125, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(455660.5625, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(496136.3750, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(389292.8125, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(272891.2500, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(289251.8750, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(73087.3594, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(137523.5156, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(46132.3516, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(38502.5234, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(33976.2461, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(43707.4648, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(21662.8145, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(13052.1143, device='c

In [ ]:
num_epochs=3
optimizer =torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.MSELoss(reduction='mean')
# Train and evaluate
model, hist = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef_net_adam1.txt')

Epoch 0/2
----------
tensor(10052.4648, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(12248.3545, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(11553.3662, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(9536.5107, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(6805.2119, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3946.8884, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4307.1206, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(11555.7344, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(6081.4370, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(9544.8711, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(6038.4731, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(7909.7773, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(9623.5273, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(7681.7686, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(13583.6074, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(6766.0176, device='cuda:0', grad_fn=<

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.001), num_epochs=3)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef_net_adam.txt')

Epoch 0/2
----------
tensor(3756.8237, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(29475.7520, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(5094.0674, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(6295.3311, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(5429.8594, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4291.2773, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1631.7502, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3508.0024, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4047.3027, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2114.8535, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3424.5869, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3621.0417, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1410.2703, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(8485.8174, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(12587.5020, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1320.9062, device='cuda:0', grad_fn=<Mse

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.001), num_epochs=3)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef_net_adam.txt')

Epoch 0/2
----------
tensor(2893.8232, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2540.3550, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2793.8267, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1273.9226, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(10621.8945, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2795.8535, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2583.6465, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2635.6509, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2483.7656, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4025.1831, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3121.3335, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(5674.1597, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1881.5269, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1358.8624, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4545.9023, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2109.8594, device='cuda:0', grad_fn=<MseL

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.001), num_epochs=3)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef_net_adam.txt')

Epoch 0/2
----------
tensor(944.5835, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2956.5759, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2518.6460, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(921.9926, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1383.4583, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(851.2384, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3022.0774, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1605.8302, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4411.5728, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2044.7336, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2150.5935, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3926.5879, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3243.9272, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(3387.2542, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1822.9192, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(15316.8076, device='cuda:0', grad_fn=<MseLoss

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.001), num_epochs=4)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef_net_adam__.txt')

Epoch 0/3
----------
tensor(1369.0421, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2469.8008, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(816.6846, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1022.6912, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1558.9275, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1719.9197, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(870.5128, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1680.1464, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(4071.4688, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1899.3228, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(6213.9316, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(841.3873, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(526.0975, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1590.5012, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1461.2119, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(2190.3359, device='cuda:0', grad_fn=<MseLossBa

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.0003), num_epochs=8)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef5_net_adam2.txt')

Epoch 0/7
----------
tensor(1021.0389, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1839.2002, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(974.2304, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(787.0370, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(957.6029, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(837.1511, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(605.2919, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1114.9446, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(810.6183, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1328.9108, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(821.8687, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1304.4150, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(844.1669, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1700.6825, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(775.7939, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1122.0782, device='cuda:0', grad_fn=<MseLossBackwar

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.0001), num_epochs=8)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef5_net_adam3.txt')

Epoch 0/7
----------
tensor(1893.9414, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(491.1410, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(407.0396, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(451.3976, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(591.7089, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(620.9421, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(514.6200, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(685.8617, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1161.3086, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1103.7693, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1507.6575, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(894.1885, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(757.1544, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(611.4003, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(863.7394, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(524.9700, device='cuda:0', grad_fn=<MseLossBackward>)

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.00003), num_epochs=8)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef5_net_adam4.txt')

Epoch 0/7
----------
tensor(557.4513, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1275.9635, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1142.1570, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(701.0946, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(473.2738, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(707.8235, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(559.0863, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(639.2803, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(572.5847, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(675.5094, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1098.4247, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(776.4652, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(883.4783, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(604.8850, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(305.4053, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1144.2603, device='cuda:0', grad_fn=<MseLossBackward>)

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.00001), num_epochs=5)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef5_net_adam5.txt')

Epoch 0/4
----------
tensor(474.4205, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(520.7783, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(691.4502, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1915.4984, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(981.7435, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(660.6892, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(725.6124, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(658.2329, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(616.6148, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(665.7450, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(538.1611, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(398.0583, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1031.8822, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(390.9847, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(505.6226, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(610.5330, device='cuda:0', grad_fn=<MseLossBackward>)
t

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.000003), num_epochs=5)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef5_net_adam6.txt')

Epoch 0/4
----------
tensor(459.1544, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(959.9667, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(755.1567, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(504.5496, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(783.6774, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(405.2543, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1557.6870, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(697.2979, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(538.7745, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(662.6500, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(566.3698, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(542.2593, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(756.8997, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(754.3699, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(590.2037, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(314.1022, device='cuda:0', grad_fn=<MseLossBackward>)
te

In [ ]:
model, hist = train_model(model, dataloaders_dict, criterion, torch.optim.Adam(model.parameters(), lr=0.000001), num_epochs=5)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_ef5_net_adam6.txt')

Epoch 0/4
----------
tensor(374.0327, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(840.9709, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(1089.3677, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(248.8978, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(857.8651, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(619.9756, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(376.7505, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(801.1678, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(324.1050, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(639.1308, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(641.3213, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(631.1763, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(369.9946, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(892.0739, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(471.9645, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(931.4033, device='cuda:0', grad_fn=<MseLossBackward>)
te

# Prediction and data uploading

In [ ]:
A=[[i for i in range(10000)],[0]*10000]
df=pd.DataFrame(A).transpose()
df.columns=['ImageID','label']
i=0
for f in os.listdir('data/test/'):
  im=Image.open('data/test/'+f)
  a=trainTrans(im)
  tens=torch.reshape(a,(1,3,a.size(1),a.size(2)))
  inputs = tens.to(device)
  outputs = (model(inputs).detach().cpu().numpy())
  df.iloc[int(f.split('.')[0]),1]=outputs[0][0]
df.to_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv',index=False)
!aicrowd submission create -c f1-speed-recognition -f '/content/drive/MyDrive/Colab Notebooks/submission.csv'